In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join('..', 'src')))

from funkwpap import *
import sympy as sp, pandas as pd, numpy as np, tqdm, time, sys, matplotlib.pyplot as plt
from scipy.optimize import fsolve
from random import random
from scipy.optimize import minimize
from gurobipy import *

In [2]:
pp = sp.symbols('pp') # permit price
q1, q2 = sp.symbols('q1 q2') # quantity of firm1 and firm2
x1, x2 = sp.symbols('x1 x2') # emission of firm1 and firm2
cap = 50 # total emission cap

profit1 = (10 - 0.1 * (q1 + q2)) * q1 - 2*(q1 - x1)**2 - pp*x1
profit2 = (10 - 0.1 * (q1 + q2)) * q2 - 100*(q2 - x2)**2 - pp*x2

# First order conditions
foc1 = sp.diff(profit1, q1)
foc2 = sp.diff(profit2, q2)
foc3 = sp.diff(profit1, x1)
foc4 = sp.diff(profit2, x2)

# Solve for q1, q2, x1, x2
solutions = sp.solve([foc1, foc2, foc3, foc4, x1 + x2 - cap], (q1, q2, x1, x2, pp))

print(solutions)

#Now again using a least squares approach and scipy optimization
print(foc1,"|", foc2,"|", foc3,"|", foc4)






{pp: 2.40789790512882, q1: 25.3070069829039, q2: 25.3070069829039, x1: 24.7050325066217, x2: 25.2949674933783}
-4.2*q1 - 0.1*q2 + 4*x1 + 10 | -0.1*q1 - 200.2*q2 + 200*x2 + 10 | -pp + 4*q1 - 4*x1 | -pp + 200*q2 - 200*x2


Παρακάτω λύνουμε το ίδιο βάζοντας ουσιαστικά κάθε μεταβλητή που μας ενδιαφέρει στο τετράγωνο. και ελαχιστοποιώντας το άθροισμά του. 

In [3]:
example = Model('example')
q1 = example.addVar(vtype=GRB.CONTINUOUS, name='q1', lb=0)
q2 = example.addVar(vtype=GRB.CONTINUOUS, name='q2', lb=0)
x1 = example.addVar(vtype=GRB.CONTINUOUS, name='x1', lb=0)
x2 = example.addVar(vtype=GRB.CONTINUOUS, name='x2', lb=0)
pp = example.addVar(vtype=GRB.CONTINUOUS, name='pp', lb=0)
foc1 = -4.2*q1 - 0.1*q2 + 4*x1 + 10
foc2 = -0.1*q1 - 200.2*q2 + 200*x2 + 10
foc3 = -pp + 4*q1 - 4*x1
foc4 = -pp + 200*q2 - 200*x2
obj_fn = foc1**2 + foc2**2 + foc3**2 + foc4**2 + (cap - x1 - x2)**2
example.setObjective(obj_fn, GRB.MINIMIZE)

example.addConstr(x1 + x2 <= cap)
example.addConstr(q1 >= x1)
example.addConstr(q2 >= x2)

example.optimize()

for v in example.getVars():
    print('%s: %g' % (v.varName, v.x))

Set parameter Username
Academic license - for non-commercial use only - expires 2025-07-02
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i7-7700HQ CPU @ 2.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 3 rows, 5 columns and 6 nonzeros
Model fingerprint: 0x0c40551a
Model has 15 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 4e+03]
  QObjective range [2e+00, 3e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+01, 5e+01]
Presolve time: 0.01s
Presolved: 3 rows, 5 columns, 6 nonzeros
Presolved model has 15 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 4
 AA' NZ     : 2.000e+01
 Factor NZ  : 2.800e+01
 Factor Ops : 1.400e+02 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual


Εδώ θα κάνουμε το ίδιο, αλλά αυτήν την φορά θα κάνουμε την δική μου ιδέα με τα constrains και το αθροισμα αλλού.

In [4]:
example = Model('example')
q1 = example.addVar(vtype=GRB.CONTINUOUS, name='q1', lb=0)
q2 = example.addVar(vtype=GRB.CONTINUOUS, name='q2', lb=0)
x1 = example.addVar(vtype=GRB.CONTINUOUS, name='x1', lb=0)
x2 = example.addVar(vtype=GRB.CONTINUOUS, name='x2', lb=0)
pp = example.addVar(vtype=GRB.CONTINUOUS, name='pp', lb=0)
foc1 = -4.2*q1 - 0.1*q2 + 4*x1 + 10
foc2 = -0.1*q1 - 200.2*q2 + 200*x2 + 10
foc3 = -pp + 4*q1 - 4*x1
foc4 = -pp + 200*q2 - 200*x2
obj_fn =   x1 + x2
example.setObjective(obj_fn, GRB.MAXIMIZE)
example.addConstr(x1 + x2 <= cap)
example.addConstr(q1 >= x1)
example.addConstr(q2 >= x2)
example.addConstr(foc1 == 0)
example.addConstr(foc2 == 0)
example.addConstr(foc3 == 0)
example.addConstr(foc4 == 0)

example.optimize()

for v in example.getVars():
    print('%s: %g' % (v.varName, v.x))

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i7-7700HQ CPU @ 2.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 7 rows, 5 columns and 18 nonzeros
Model fingerprint: 0x37dfe2d5
Coefficient statistics:
  Matrix range     [1e-01, 2e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 5e+01]
Presolve removed 2 rows and 2 columns
Presolve time: 0.01s
Presolved: 5 rows, 3 columns, 12 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.0000000e+01   1.251145e+00   0.000000e+00      0s
       1    5.0000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.02 seconds (0.00 work units)
Optimal objective  5.000000000e+01
q1: 25.307
q2: 25.307
x1: 24.705
x2: 25.295
pp: 2.4079


In [5]:
example = Model('example')
q1 = example.addVar(vtype=GRB.CONTINUOUS, name='q1', lb=0)
q2 = example.addVar(vtype=GRB.CONTINUOUS, name='q2', lb=0)
x1 = example.addVar(vtype=GRB.CONTINUOUS, name='x1', lb=0)
x2 = example.addVar(vtype=GRB.CONTINUOUS, name='x2', lb=0)
pp = example.addVar(vtype=GRB.CONTINUOUS, name='pp', lb=0)
foc1 = -4.2*q1 - 0.1*q2 + 4*x1 + 10
foc2 = -0.1*q1 - 200.2*q2 + 200*x2 + 10
foc3 = -pp + 4*q1 - 4*x1
foc4 = -pp + 200*q2 - 200*x2
obj_fn =   (10 - 0.1 * (q1 + q2)) * q1 - 2*(q1 - x1)**2 - pp*x1
example.setObjective(obj_fn, GRB.MAXIMIZE)
example.addConstr(x1 + x2 <= cap)
example.addConstr(q1 >= x1)
example.addConstr(q2 >= x2)
example.addConstr(foc1 == 0)
example.addConstr(foc2 == 0)
example.addConstr(foc3 == 0)
example.addConstr(foc4 == 0)

example.optimize()

for v in example.getVars():
    print('%s: %g' % (v.varName, v.x))

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i7-7700HQ CPU @ 2.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 7 rows, 5 columns and 18 nonzeros
Model fingerprint: 0xff123a72
Model has 5 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e-01, 2e+02]
  Objective range  [1e+01, 1e+01]
  QObjective range [2e-01, 8e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 5e+01]

Continuous model is non-convex -- solving as a MIP

Presolve removed 2 rows and 0 columns
Presolve time: 0.00s
Presolved: 12 rows, 11 columns, 35 nonzeros
Presolved model has 2 quadratic constraint(s)
Presolved model has 3 bilinear constraint(s)
Variable types: 11 continuous, 0 integer (0 binary)
Found heuristic solution: objective 64.7692068

Root relaxation: objective 5.187390e+03, 11 iterations, 0.00 seconds (0.00 work units)

 

In [6]:
# How would my optimzer handle this?
x = sp.symbols('x')
Regulator6 = Regulator("test1", permit_price = 2.4079, emission_cap = 50)
sector1 = Sector("sector1", 10-0.1*x, 0, Regulator6)
fra = Country("France", 50, Regulator6)
# Create Firms using objects
firm1 = Firm('firm1', 1, 1, x*0, 2*x**2 , 0, 0, 0, regulator= Regulator6)
firm2 = Firm('firm2', 1, 1, x*0, 100*x**2 , 0, 0, 0, regulator= Regulator6)

Regulator6.find_optimal_permit_price_to_meet_the_emission_cap_requirements(precision=0.0001)
for firm in Regulator6.firm_registry.values():
    print(firm.name, firm.actual_output, firm.emission)

Permit price: 2.441406 price = 2.4, cap = 50, second_stage = False, a = 1.0000Permit price: 2.44140625 and total emission: 48.92825602716198 and emission cap 50
firm1 24.766079242393324 24.163176784768652
firm2 24.766079242393324 24.765079242393323
